In [147]:
import feather
# Load the DataFrame from the Feather file
df_routes = feather.read_dataframe('data_with_responses.feather')

In [148]:
import pandas as pd

# Specify the path to your Excel file
excel_file_path = r'C:\Users\Doktor\Documents\GitHub\Bridges\Data\routes_selected_new.xlsx'
survey_file_path = r'C:\Users\Doktor\Documents\GitHub\Bridges\Data\surveys\Survey.csv'
bridges_path = r'C:\Users\Doktor\Documents\GitHub\Bridges\Data\Bridges_processed.csv'
# Load the Excel file into a DataFrame
df = pd.read_excel(excel_file_path)
df_survey = pd.read_csv(survey_file_path, encoding='utf-8', delimiter=',')
df_bridges_all = pd.read_csv(bridges_path, encoding='utf-8', delimiter=',')

In [149]:
# Merge the two DataFrames based on the 'ID_mosta' column
df_routes = df_routes.merge(
    df[['ID_mosta', 'ID_úseku']], on='ID_mosta', how='left')

In [150]:
df_survey['ÚSEK'] = df_survey['ÚSEK'].astype(str)
df_routes['ID_úseku'] = df_routes['ID_úseku'].astype(str).str.zfill(5)

In [151]:
# Change the value of ["ID_úseku"] at index 75
df_routes.at[75, 'ID_úseku'] = "01030"


# Merge df_survey with df_routes based on 'ID_úseku' and 'CESTA'
df_routes = df_routes.merge(
    df_survey[['ÚSEK', 'T', 'O', 'M']], left_on='ID_úseku', right_on='ÚSEK', how='left')

# Drop the 'CESTA' column from the merged DataFrame, as it's redundant
df_routes.drop(columns=['ÚSEK'], inplace=True)

In [152]:
# Specify the columns you want to add from df_bridges_all
columns_to_add = ['Trieda_PK', 'Názov_mosta', 'Druh_konštrukcie', 'Material',
                  'Predmet_premostenia', 'Rok_postavenia', 'Dĺžka_premostenia',
                  'Volná_šírka_mosta', 'n_2012', 'n_2022', 'n_2023']

# Merge df_routes with df_bridges_all based on 'ID_mosta'
df_routes = df_routes.merge(df_bridges_all[columns_to_add + ['ID_mosta']],
                            on='ID_mosta', how='left')

In [153]:
df_bridges_all.columns

Index(['ID_mosta', 'Trieda_PK', 'Číslo_PK', 'Most_DC', 'Názov_mosta', 'ID_DC',
       'správcovské_číslo', 'Zemepisná_dĺžka', 'Zemepisná_šírka', 'Výška',
       'Druh_konštrukcie', 'Material', 'Normálna_zaťažiteľnosť',
       'Výhradná_zaťažiteľnosť_t', 'Výnimočná_zaťažiteľnosť_t',
       'Spôsob_stanovenia', 'Predmet_premostenia', 'Rok_postavenia',
       'Zaťažovacia_trieda_STN', 'Počet_otvorov', 'Dĺžka_premostenia',
       'Dĺžka_nosnej_konštrukcie_m', 'Volná_šírka_mosta',
       'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2',
       'Plocha_mosta_m2', 'Vlastník_úseku', 'Správca_úseku', 'Okres',
       'Smer_úseku', 'Typ_úseku', 'Počiatočný_uzol', 'Koncový_uzol', 'úsek_ID',
       'úsekové_staničenie_začiatku_DC_m',
       'Kilometrovníkové_staničenie_začiatku_DC_km',
       'Kumulatívne_staničenie_začiatku_DC_km', 'Poradie', 'Kraj', 'n_2012',
       'n_2013', 'n_2014', 'n_2015', 'n_2016', 'n_2017', 'n_2018', 'n_2019',
       'n_2020', 'n_2021', 'n_2022', 'n_2023'],
      

In [154]:
df_routes.columns

Index(['ID_mosta', 'Okres', 'Custom_coor', 'br_lat', 'br_long', 'start_lat',
       'start_long', 'end_lat', 'end_long', 'Hexagon_Coordinates', 'car_req_1',
       'car_req_alt', 'car_resp_1', 'car_resp_alt', 'ID_úseku', 'T', 'O', 'M',
       'Trieda_PK', 'Názov_mosta', 'Druh_konštrukcie', 'Material',
       'Predmet_premostenia', 'Rok_postavenia', 'Dĺžka_premostenia',
       'Volná_šírka_mosta', 'n_2012', 'n_2022', 'n_2023'],
      dtype='object')

In [155]:
df_routes.iloc[75]

ID_mosta                                                           M4087
Okres                                                           Bardejov
Custom_coor                                                         None
br_lat                                                          49.29704
br_long                                                         21.25675
start_lat                                                      49.295154
start_long                                                     21.267516
end_lat                                                        49.302671
end_long                                                        21.23824
Hexagon_Coordinates    [[21.257025498706447, 49.29704], [21.256887749...
car_req_1              {\n  "points": [\n    [\n      21.267516,\n   ...
car_req_alt            {\n  "points": [\n    [\n      21.267516,\n   ...
car_resp_1             {'hints': {'visited_nodes.average': 170.0, 'vi...
car_resp_alt           {'hints': {'visited_nodes.av

In [156]:
def calculate_car_time_1(row):
    if row is not None:
        return row['paths'][0]['time'] / 1000 / 60
    else:
        return None


def calculate_car_time_alt(row):
    if row is not None:
        return row['paths'][0]['time'] / 1000 / 60
    else:
        return None


def calculate_car_dist_1(row):
    if row is not None:
        return row['paths'][0]['distance'] / 1000
    else:
        return None


def calculate_car_dist_alt(row):
    if row is not None:
        return row['paths'][0]['distance'] / 1000
    else:
        return None


# Calculate and store car_time_1, car_time_alt, car_dist_1, and car_dist_alt
df_routes['car_time_1'] = df_routes['car_resp_1'].apply(calculate_car_time_1)
df_routes['car_time_alt'] = df_routes['car_resp_alt'].apply(
    calculate_car_time_alt)
df_routes['car_dist_1'] = df_routes['car_resp_1'].apply(calculate_car_dist_1)
df_routes['car_dist_alt'] = df_routes['car_resp_alt'].apply(
    calculate_car_dist_alt)

In [157]:
df_routes['time_diff'] = df_routes['car_time_alt'] - df_routes['car_time_1']
df_routes['dist_diff'] = df_routes['car_dist_alt'] - df_routes['car_dist_1']

In [158]:
df_routes_time = df_routes.sort_values(by='time_diff', ascending=False)
df_routes_time

,ID_mosta,Okres,Custom_coor,br_lat,br_long,start_lat,start_long,end_lat,end_long,Hexagon_Coordinates,...,Volná_šírka_mosta,n_2012,n_2022,n_2023,car_time_1,car_time_alt,car_dist_1,car_dist_alt,time_diff,dist_diff
111,M5316,Medzilaborce,None,49.283098,21.955800,49.269547,21.992666,49.271736,21.902402,"[[21.956075504805312, 49.283097923], [21.95593...",...,8.6,4,6,6,10.419700,91.818450,8.387578,93.369178,81.398750,84.981600
99,M3386,Gelnica,None,48.878809,20.997179,48.853712,20.940274,48.892571,21.013551,"[[20.997451776847683, 48.878808695], [20.99731...",...,11.2,3,6,6,10.313933,84.142200,7.918419,34.557649,73.828267,26.639230
67,M4240,Humenné,None,48.928456,21.912401,48.931729,21.957430,48.930384,21.874145,"[[21.91267470937467, 48.928456086], [21.912537...",...,16.0,3,5,6,8.612083,64.007533,7.022536,51.156541,55.395450,44.134005
62,M1195,Rimavská Sobota,None,48.661440,19.941654,48.683924,19.941162,48.571793,19.954388,"[[19.941926411751933, 48.661439887], [19.94179...",...,9.5,3,3,5,13.449267,63.777967,13.007548,64.137117,50.328700,51.129569
58,M6289,Stará Ľubovňa,None,49.306690,20.686370,49.302170,20.687249,49.302824,20.641968,"[[20.686645552661037, 49.30669], [20.686507776...",...,10.25,2,5,5,5.187250,53.424200,4.113475,51.394764,48.236950,47.281289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,M5723,Detva,None,48.545846,19.334897,48.544268,19.399982,48.549308,19.304028,"[[19.335168411920563, 48.545845803], [19.33503...",...,11.0,4,6,6,6.485667,6.379583,7.346073,8.607793,-0.106083,1.261720
138,M6054,Hlohovec,None,48.366082,17.736201,48.284953,17.733761,48.435454,17.768358,"[[17.73647135769861, 48.366081596], [17.736336...",...,10.0,5,6,6,23.404483,22.060483,18.355872,33.845871,-1.344000,15.489999
94,M7088,Spišská Nová Ves,None,48.916991,20.582879,48.909656,20.584604,48.941746,20.576181,"[[20.583152396604714, 48.916991], [20.58301569...",...,5.0,4,5,6,8.140600,NaN,5.181574,NaN,NaN,NaN
107,M3011,Bratislava I,None,48.162426,17.097242,48.168682,17.076438,48.159364,17.104073,"[[17.097511269343144, 48.162426001], [17.09737...",...,12.75,4,5,5,3.126500,NaN,2.373473,NaN,NaN,NaN


In [159]:
import feather

# Save your DataFrame to a Feather file
feather.write_dataframe(df_routes, 'data_with_times.feather')

In [226]:
df_routes.columns

Index(['ID_mosta', 'Okres', 'Custom_coor', 'br_lat', 'br_long', 'start_lat',
       'start_long', 'end_lat', 'end_long', 'Hexagon_Coordinates', 'car_req_1',
       'car_req_alt', 'car_resp_1', 'car_resp_alt', 'ID_úseku', 'T', 'O', 'M',
       'Trieda_PK', 'Názov_mosta', 'Druh_konštrukcie', 'Material',
       'Predmet_premostenia', 'Rok_postavenia', 'Dĺžka_premostenia',
       'Volná_šírka_mosta', 'n_2012', 'n_2022', 'n_2023', 'car_time_1',
       'car_time_alt', 'car_dist_1', 'car_dist_alt', 'time_diff', 'dist_diff'],
      dtype='object')

In [193]:
df_routes_time["time_diff"]

67     55.395450
58     48.236950
40     44.909517
87     35.436183
29     33.616000
         ...    
20     -0.106083
138    -1.344000
94           NaN
107          NaN
114          NaN
Name: time_diff, Length: 148, dtype: float64

In [199]:
import plotly.express as px

# Sort the DataFrame by the "time_diff" column in ascending order
df_sorted = df_routes_time.sort_values(by='time_diff', ascending=True)

# Create a bar plot using Plotly Express
fig = px.bar(
    df_sorted,
    x=df_sorted.index,  # Use the DataFrame index as x values
    y='time_diff',
    labels={'x': 'Index', 'y': 'Time Difference (minutes)'},
    title='Time Difference for Bridges (Sorted)',
)

# Customize the layout
fig.update_xaxes(type='category')
fig.update_xaxes(tickangle=45)
fig.update_layout(width=800, height=600)

# Show the interactive plot
fig.show()

In [161]:
# Calculate the mean of time_diff
mean_time_diff = df_routes_time['time_diff'].mean()

# Calculate the median of time_diff
median_time_diff = df_routes_time['time_diff'].median()

print("Mean Time Difference:", mean_time_diff)
print("Median Time Difference:", median_time_diff)

Mean Time Difference: 10.021827241379311
Median Time Difference: 6.581149999999997


In [286]:
import plotly.express as px
import numpy as np

# Variables for customization
axis_font_size = 11
axis_tick_font_size = 12
title_font_size = 20
bg_color = "#FFFFFF"
fig_color = "#eeeeee"
bar_color = "#FF6600"

# Sort the DataFrame by the "time_diff" column in ascending order and reset the index
df_sorted = df_routes_time.sort_values(
    by='time_diff', ascending=True).reset_index(drop=True)

# Calculate the mean value of the "time_diff" column
mean_time_diff = np.mean(df_sorted['time_diff'])

# Create a bar plot using Plotly Express
fig = px.bar(
    df_sorted,
    x=df_sorted.index,
    y='time_diff',
    title='Časový rozdiel',
)

# Update the layout to tighten the margins
fig.update_layout(
    plot_bgcolor=fig_color,
    paper_bgcolor=bg_color,
    xaxis=dict(
        showticklabels=False,
        tickfont=dict(size=axis_tick_font_size),
    ),
    yaxis=dict(
        titlefont=dict(size=axis_font_size),
        tickfont=dict(size=axis_tick_font_size),
    ),
    titlefont=dict(size=title_font_size),
    xaxis_title="",
    yaxis_title="Čas [min]",
    margin=dict(l=65, r=25, t=75, b=15),
)

# Customize bar color
fig.update_traces(marker_color=bar_color)

# Set size of the figure
fig.update_layout(width=700, height=450)

# Add a horizontal line for the mean value with dashes
fig.add_shape(
    dict(
        type='line',
        x0=0, x1=len(df_sorted),  # Start and end positions of the line
        y0=mean_time_diff, y1=mean_time_diff,  # Mean value for y
        # Customize line color, width, and dash style
        line=dict(color='#444444', width=2, dash='dash'),
    )
)

# Add a custom annotation for the mean value
fig.add_annotation(
    text=f'Priemer: {mean_time_diff:.2f} min',  # Display the mean value
    x=25,  # Adjust the x position to place the annotation at the left
    # Adjust the y position to place the annotation above the plot (you may need to adjust this value)
    y=12.5,
    showarrow=False,  # Hide the arrow
    font=dict(size=axis_font_size),  # Customize font size
)



# Show the interactive plot
fig.show()

In [289]:
df_sorted_2 = df_routes_time.sort_values(
    by='time_diff', ascending=False).reset_index(drop=True)

df_sorted_2

,ID_mosta,Okres,Custom_coor,br_lat,br_long,start_lat,start_long,end_lat,end_long,Hexagon_Coordinates,...,n_2012,n_2022,n_2023,car_time_1,car_time_alt,car_dist_1,car_dist_alt,time_diff,dist_diff,O_times_time_diff
0,M5316,Medzilaborce,None,49.283098,21.955800,49.269547,21.992666,49.271736,21.902402,"[[21.956075504805312, 49.283097923], [21.95593...",...,4,6,6,10.419700,91.818450,8.387578,93.369178,81.398750,84.981600,1.831472e+04
1,M3386,Gelnica,None,48.878809,20.997179,48.853712,20.940274,48.892571,21.013551,"[[20.997451776847683, 48.878808695], [20.99731...",...,3,6,6,10.313933,84.142200,7.918419,34.557649,73.828267,26.639230,3.352542e+05
2,M4240,Humenné,None,48.928456,21.912401,48.931729,21.957430,48.930384,21.874145,"[[21.91267470937467, 48.928456086], [21.912537...",...,3,5,6,8.612083,64.007533,7.022536,51.156541,55.395450,44.134005,1.013571e+06
3,M1195,Rimavská Sobota,None,48.661440,19.941654,48.683924,19.941162,48.571793,19.954388,"[[19.941926411751933, 48.661439887], [19.94179...",...,3,3,5,13.449267,63.777967,13.007548,64.137117,50.328700,51.129569,1.457519e+05
4,M6289,Stará Ľubovňa,None,49.306690,20.686370,49.302170,20.687249,49.302824,20.641968,"[[20.686645552661037, 49.30669], [20.686507776...",...,2,5,5,5.187250,53.424200,4.113475,51.394764,48.236950,47.281289,7.234578e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,M5723,Detva,None,48.545846,19.334897,48.544268,19.399982,48.549308,19.304028,"[[19.335168411920563, 48.545845803], [19.33503...",...,4,6,6,6.485667,6.379583,7.346073,8.607793,-0.106083,1.261720,-4.494751e+02
144,M6054,Hlohovec,None,48.366082,17.736201,48.284953,17.733761,48.435454,17.768358,"[[17.73647135769861, 48.366081596], [17.736336...",...,5,6,6,23.404483,22.060483,18.355872,33.845871,-1.344000,15.489999,-3.592512e+03
145,M7088,Spišská Nová Ves,None,48.916991,20.582879,48.909656,20.584604,48.941746,20.576181,"[[20.583152396604714, 48.916991], [20.58301569...",...,4,5,6,8.140600,NaN,5.181574,NaN,NaN,NaN,NaN
146,M3011,Bratislava I,None,48.162426,17.097242,48.168682,17.076438,48.159364,17.104073,"[[17.097511269343144, 48.162426001], [17.09737...",...,4,5,5,3.126500,NaN,2.373473,NaN,NaN,NaN,NaN


In [283]:
import plotly.express as px
import numpy as np

# Variables for customization
axis_font_size = 11
axis_tick_font_size = 12
title_font_size = 20
bg_color = "#FFFFFF"
fig_color = "#eeeeee"
bar_color = "#FF6600"

# Sort the DataFrame by the "dist_diff" column in ascending order and reset the index
df_sorted = df_routes_time.sort_values(
    by='dist_diff', ascending=True).reset_index(drop=True)

# Calculate the mean value of the "dist_diff" column
mean_dist_diff = np.mean(df_sorted['dist_diff'])

# Create a bar plot using Plotly Express
fig = px.bar(
    df_sorted,
    x=df_sorted.index,
    y='dist_diff',
    title='Predĺženie trasy',
)

# Update the layout to tighten the margins
fig.update_layout(
    plot_bgcolor=fig_color,
    paper_bgcolor=bg_color,
    xaxis=dict(
        showticklabels=False,
        tickfont=dict(size=axis_tick_font_size),
    ),
    yaxis=dict(
        titlefont=dict(size=axis_font_size),
        tickfont=dict(size=axis_tick_font_size),
    ),
    titlefont=dict(size=title_font_size),
    xaxis_title="",
    yaxis_title="Vzdialenosť [km]",
    margin=dict(l=65, r=25, t=75, b=15),
)

# Customize bar color
fig.update_traces(marker_color=bar_color)

# Set size of the figure
fig.update_layout(width=700, height=450)

# Add a horizontal line for the mean value with dashes
fig.add_shape(
    dict(
        type='line',
        x0=0, x1=len(df_sorted),  # Start and end positions of the line
        y0=mean_dist_diff, y1=mean_dist_diff,  # Mean value for y
        # Customize line color, width, and dash style
        line=dict(color='#444444', width=2, dash='dash'),
    )
)

# Add a custom annotation for the mean value
fig.add_annotation(
    text=f'Priemer: {mean_dist_diff:.2f} min',  # Display the mean value
    x=25,  # Adjust the x position to place the annotation at the left
    # Adjust the y position to place the annotation above the plot (you may need to adjust this value)
    y=7.56 + 2.5,
    showarrow=False,  # Hide the arrow
    font=dict(size=axis_font_size),  # Customize font size
)


# Show the interactive plot
fig.show()

In [163]:
# Calculate the mean of dist_diff
mean_dist_diff = df_routes_time['dist_diff'].mean()

# Calculate the median of dist_diff
median_dist_diff = df_routes_time['dist_diff'].median()

print("Mean Distance Difference:", mean_dist_diff)
print("Median Distance Difference:", median_dist_diff)

Mean Distance Difference: 7.562527006896553
Median Distance Difference: 4.007014999999999


In [164]:
import plotly.express as px

# Multiply the "O" column by the "time_diff" column
df_routes_time['O_times_time_diff'] = df_routes_time['O'] * \
    df_routes_time['time_diff']

# Create a bar plot using Plotly Express
fig = px.bar(
    df_routes_time,
    x='ID_mosta',
    y='O_times_time_diff',  # Use the new column
    labels={'x': 'Bridge ID', 'y': 'O * Time Difference'},
    title='O * Time Difference for Bridges'
)

# Customize the layout
fig.update_xaxes(type='category', categoryorder='total ascending')
fig.update_xaxes(tickangle=45)
fig.update_layout(width=800, height=600)

# Show the interactive plot
fig.show()

In [166]:
df_routes_time = df_routes_time.sort_values(
    by='O_times_time_diff', ascending=False)
df_routes_time

,ID_mosta,Okres,Custom_coor,br_lat,br_long,start_lat,start_long,end_lat,end_long,Hexagon_Coordinates,...,n_2012,n_2022,n_2023,car_time_1,car_time_alt,car_dist_1,car_dist_alt,time_diff,dist_diff,O_times_time_diff
67,M4240,Humenné,None,48.928456,21.912401,48.931729,21.957430,48.930384,21.874145,"[[21.91267470937467, 48.928456086], [21.912537...",...,3,5,6,8.612083,64.007533,7.022536,51.156541,55.395450,44.134005,1.013571e+06
58,M6289,Stará Ľubovňa,None,49.306690,20.686370,49.302170,20.687249,49.302824,20.641968,"[[20.686645552661037, 49.30669], [20.686507776...",...,2,5,5,5.187250,53.424200,4.113475,51.394764,48.236950,47.281289,7.234578e+05
40,M1514,Komárno,None,47.762158,18.143206,47.766700,18.154460,47.765664,18.091368,"[[18.14347349132844, 47.76215808], [18.1433398...",...,5,6,6,8.195683,53.105200,5.219593,45.815349,44.909517,40.595756,7.081333e+05
87,M6702,Hlohovec,None,48.430096,17.791829,48.425339,17.803328,48.439491,17.712535,"[[17.792099859197307, 48.430096307], [17.79196...",...,4,5,6,7.994517,43.430700,7.158568,44.692739,35.436183,37.534171,5.506074e+05
29,M3265,Banská Bystrica,None,48.778806,19.124916,48.790308,19.103030,48.737204,19.140441,"[[19.125188498178176, 48.778806376], [19.12505...",...,4,5,5,10.217700,43.833700,7.963645,19.035463,33.616000,11.071818,3.854747e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,M5723,Detva,None,48.545846,19.334897,48.544268,19.399982,48.549308,19.304028,"[[19.335168411920563, 48.545845803], [19.33503...",...,4,6,6,6.485667,6.379583,7.346073,8.607793,-0.106083,1.261720,-4.494751e+02
138,M6054,Hlohovec,None,48.366082,17.736201,48.284953,17.733761,48.435454,17.768358,"[[17.73647135769861, 48.366081596], [17.736336...",...,5,6,6,23.404483,22.060483,18.355872,33.845871,-1.344000,15.489999,-3.592512e+03
94,M7088,Spišská Nová Ves,None,48.916991,20.582879,48.909656,20.584604,48.941746,20.576181,"[[20.583152396604714, 48.916991], [20.58301569...",...,4,5,6,8.140600,NaN,5.181574,NaN,NaN,NaN,NaN
107,M3011,Bratislava I,None,48.162426,17.097242,48.168682,17.076438,48.159364,17.104073,"[[17.097511269343144, 48.162426001], [17.09737...",...,4,5,5,3.126500,NaN,2.373473,NaN,NaN,NaN,NaN


In [168]:
df_routes_analyze = df_routes_time.copy()

In [169]:
df_routes_analyze.columns

Index(['ID_mosta', 'Okres', 'Custom_coor', 'br_lat', 'br_long', 'start_lat',
       'start_long', 'end_lat', 'end_long', 'Hexagon_Coordinates', 'car_req_1',
       'car_req_alt', 'car_resp_1', 'car_resp_alt', 'ID_úseku', 'T', 'O', 'M',
       'Trieda_PK', 'Názov_mosta', 'Druh_konštrukcie', 'Material',
       'Predmet_premostenia', 'Rok_postavenia', 'Dĺžka_premostenia',
       'Volná_šírka_mosta', 'n_2012', 'n_2022', 'n_2023', 'car_time_1',
       'car_time_alt', 'car_dist_1', 'car_dist_alt', 'time_diff', 'dist_diff',
       'O_times_time_diff'],
      dtype='object')

In [170]:
columns_to_drop = ['Custom_coor', 'Hexagon_Coordinates',
                   'car_req_1', 'car_req_alt', 'car_resp_1', 'car_resp_alt', 'O_times_time_diff']
df_routes_analyze = df_routes_analyze.drop(columns=columns_to_drop)

In [172]:
file_path = 'df_routes_analyze.xlsx'

# Use the to_excel() method to save the DataFrame
df_routes_analyze.to_excel(file_path, index=False)

In [285]:
bridge_m5316 = df_routes[df_routes['ID_mosta'] == 'M3011']
print(bridge_m5316['car_req_alt'].item())

{
  "points": [
    [
      17.076438,
      48.168682
    ],
    [
      17.104073,
      48.159364
    ]
  ],
  "profile": "car",
  "points_encoded": false,
  "ch.disable": true,
  "custom_model": {
    "distance_influence": 15,
    "priority": [
      {
        "if": "in_area1",
        "multiply_by": 0
      }
    ],
    "speed": [],
    "areas": {
      "area1": {
        "type": "Feature",
        "geometry": {
          "type": "Polygon",
          "coordinates": [
            [
              [
                17.097511269343144,
                48.162426001
              ],
              [
                17.097376593671573,
                48.16258159377133
              ],
              [
                17.09710724232843,
                48.16258159377133
              ],
              [
                17.09697256665686,
                48.162426001
              ],
              [
                17.09710724232843,
                48.16227040822867
              ],
       